In [9]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
import copy
def grad_U(Ui, Yij, Vj, ai, bj, reg, eta, mu):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return eta * (reg * Ui - Vj*((Yij - (mu + np.dot(Ui, Vj) + ai + bj))))

def grad_V(Vj, Yij, Ui, ai, bj, reg, eta, mu):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return eta * (reg * Vj - Ui*((Yij - (mu + np.dot(Ui, Vj) + ai + bj))))

def grad_a(Ui, Yij, Vj, ai,bj, reg, eta, mu):
    """Calculate the bias term for U"""
    return eta * (reg * ai - (Yij - (mu + np.dot(Ui, Vj) + ai + bj)))

def grad_b(Ui, Yij, Vj, ai, bj, reg, eta, mu):
    """Calculate the bias term for V"""
    return eta * (reg * bj - (Yij - (mu + np.dot(Ui, Vj) + ai + bj)))

def get_err(U, V, Y, a, b, mu, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    errors = []
    for (user, movie, rating) in Y:
        err = (rating - (mu + np.dot(U[user - 1], V[movie - 1]) + a[user - 1] + b[movie - 1]))**2
        errors.append(err)
    return  np.sqrt(sum(errors))/len(Y)

    
def train_model_with_bias(M, N, K, eta, reg, mu,  Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    # Initialize the U and V matrices to random
    # values in [-0.5, 0.5]
    U = np.random.rand(M, K) - 0.5
    V = np.random.rand(N, K) - 0.5
    
    # Have one ai for every user, and one vj for every movie
    a = np.random.rand(M, 1) - 0.5
    b = np.random.rand(N, 1) - 0.5
    
    delta_01 = None
    delta = None
    n = 0
    prev_loss = get_err(U, V, Y, a, b, mu, reg)
    
    print(prev_loss)
    # Run for input number of epochs
    while n < max_epochs:
        # Implement early stopping
        if delta_01 != None and delta != None and delta < eps * delta_01:
            break
        # Shuffle X and Y in order to add stochasticity
        p = np.random.permutation(len(Y))
        Y = Y[p]
        # In each epoch, iterate through all the points and
        # update the U and V vectors
        # Need to update the U and V simultaneously, so need to create a temp variable.
        for (user, movie, rating) in Y:
            gradu = grad_U(U[user - 1], rating, V[movie - 1], a[user - 1], b[movie - 1], reg, eta, mu)
            gradv = grad_V(V[movie - 1], rating, U[user - 1], a[user - 1], b[movie - 1], reg, eta, mu)
            grada = grad_a(U[user - 1], rating, V[movie - 1], a[user - 1], b[movie - 1], reg, eta, mu)
            gradb = grad_b(U[user - 1], rating, V[movie - 1], a[user - 1], b[movie - 1], reg, eta, mu)         
            U[user - 1] = U[user - 1] - gradu
            V[movie - 1] = V[movie - 1] - gradv
            a[user - 1] = a[user -1] - grada
            b[movie - 1] = b[movie -1] - gradb             
        if n == 0:
            curr_loss = get_err(U, V,Y, a, b, mu, reg)
            delta_01 = prev_loss - curr_loss
            prev_loss = curr_loss
        else:
            curr_loss = get_err(U, V,Y, a, b, mu, reg)
            print(curr_loss)
            delta = prev_loss - curr_loss
            prev_loss = curr_loss
        n += 1
    print(n)
    final_mse = get_err(U, V, Y, a,b, mu)
    return (U, V, a, b, final_mse)






Rahil's Code

In [5]:
def grad_U(Ui, Yij, Vj, reg, eta, ai, bj, mu):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    eta (the learning rate), ai (ith element of a), bj (jth element of b)
    and mu (average rating).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return eta * ((reg * Ui) - Vj * (Yij - mu - np.dot(Ui, Vj) - ai - bj))

def grad_V(Vj, Yij, Ui, reg, eta, ai, bj, mu):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    eta (the learning rate), ai (ith element of a), bj (jth element of b)
    and mu (average rating).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return eta * ((reg * Vj) - Ui * (Yij - mu - np.dot(Ui, Vj) - ai - bj))

def grad_a(Ui, Yij, Vj, reg, eta, ai, bj, mu):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    eta (the learning rate), ai (ith element of a), bj (jth element of b)
    and mu (average rating).

    Returns the gradient of the regularized loss function with
    respect to ai multiplied by eta.
    """
    return eta * ((reg * ai) - (Yij - mu - np.dot(Ui, Vj) - ai - bj))

def grad_b(Ui, Yij, Vj, reg, eta, ai, bj, mu):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    eta (the learning rate), ai (ith element of a), bj (jth element of b)
    and mu (average rating).

    Returns the gradient of the regularized loss function with
    respect to bj multiplied by eta.
    """
    return eta * ((reg * bj) - (Yij - mu - np.dot(Ui, Vj) - ai - bj))

def get_err(U, V, Y, a, b, mu, reg=0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j,
    user/movie matrices U and V, user/movie vectors a and b and average rating mu.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """

    reg_err = (reg/2) * (np.linalg.norm(U) ** 2 + np.linalg.norm(V) ** 2 +
                         np.linalg.norm(a) ** 2 + np.linalg.norm(b) ** 2)
    loss_err = 0
    for point in Y:
        i, j, y = point
        loss_err += (y - mu - np.dot(U[i - 1], V[j - 1]) - a[i - 1] - b[j - 1]) ** 2
    return ((reg_err + (1/2) * loss_err)/len(Y))

def train_model(M, N, K, eta, reg, Y, mu, eps=0.0001, max_epochs=30):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V and M x 1 vector a and N x 1 vector b
    such that rating Y_ij is approximated by (UV^T)_ij + a_i + b_j + mu.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, a, b, err) consisting of U, V, a, b and the
    unregularized MSE of the model.
    """
    # Randomly initialize U, V, a and b.
    U = np.random.uniform(-0.5, 0.5, (M, K))
    V = np.random.uniform(-0.5, 0.5, (N, K))
    a = np.random.uniform(-0.5, 0.5, (M, ))
    b = np.random.uniform(-0.5, 0.5, (N, ))
    # Find the initial loss.
    prev_err = get_err(U, V, Y, a, b, mu, reg)
    for epoch in range(max_epochs):
        np.random.shuffle(Y)
        for point in Y:
            # Update the rows of U and V and elements of a and b.
            i, j, y = point
            del_U = grad_U(U[i - 1], y, V[j - 1], reg, eta, a[i - 1], b[j - 1], mu)
            del_V = grad_V(V[j - 1], y, U[i - 1], reg, eta, a[i - 1], b[j - 1], mu)
            del_a = grad_a(U[i - 1], y, V[j - 1], reg, eta, a[i - 1], b[j - 1], mu)
            del_b = grad_b(U[i - 1], y, V[j - 1], reg, eta, a[i - 1], b[j - 1], mu)
            U[i - 1] = U[i - 1] - del_U
            V[j - 1] = V[j - 1] - del_V
            a[i - 1] = a[i - 1] - del_a
            b[j - 1] = b[j - 1] - del_b
        err = get_err(U, V, Y, a, b, mu, reg)
        diff = prev_err - err
        if epoch == 0:
            # Compute the initial loss reduction.
            tol = diff
        else:
            # Check if loss reduction is too small.
            if diff/tol <= eps:
                return(U, V, a, b, get_err(U, V, Y, a, b, mu))
        prev_err = err
        print(err)
    return(U, V, a, b, np.sqrt(get_err(U, V, Y, a , b, mu)))

In [6]:
train = np.loadtxt('um/all.dta').astype(int)

In [7]:
train

array([[     1,    185,   2160,      1],
       [     1,    490,   2160,      4],
       [     1,   1811,   2160,      3],
       ...,
       [458293,  17567,   1796,      1],
       [458293,  17604,   1745,      3],
       [458293,  17689,   1715,      3]])

In [8]:
indices = np.loadtxt('um/all.idx').astype(int)

KeyboardInterrupt: 

In [ ]:
y_probe = []
y_base = []
y_valid = []
y_hidden = []
y_qual = []

for i, idx in enumerate(indices):
    if idx == 1:
        y_base.append(train[i])
    elif idx == 2:
        y_valid.append(train[i])
    elif idx == 3:
        y_hidden.append(train[i])
    elif idx == 4:
        y_probe.append(train[i])
    else:
        y_qual.append(train[i])

In [ ]:
M = max(train[:,0]).astype(int) # users
N = max(train[:,1]).astype(int) # movies

print(M)

time variance

In [6]:
from sklearn import linear_model

num_users = 458293
user_time = [None] * (num_users + 1)
user_bias = [None] * (num_users + 1)

# point : (user number)       (movie number)       (date number)       (rating)
for point in y_base:
    user_num = point[0]
    rating = point[3]
    time = point[2]
    
    user_time[user_num].append([rating, time])

for user in user_time:
    if user != None:            
        X = user_time[user][:,0]
        y = user_time[user][:,1]
        reg = LinearRegression().fit(X, y)
        user_bias.append(reg.coef_)
    
    

NameError: name 'y_base' is not defined

In [ ]:
from sklearn import linear_model

# for movie-rating hi my name is ally boobs! 

num_movies = 17770
movie_time = [None] * (num_users + 1)
movie_bias = [None] * (num_users + 1)

# point : (user number)       (movie number)       (date number)       (rating)
for point in y_base:
    movie_num = point[0]
    rating = point[3]
    time = point[2]
    
    movie_time[user_num].append([rating, time])

for user in user_time:
    if user != None:            
        X = movie_time[user][:,0]
        y = movie_time[user][:,1]
        reg = LinearRegression().fit(X, y)
        movie_bias.append(reg.coef_)
    
    

In [11]:
print("Factorizing with ", M, " users, ", N, " movies.")
k = 20
reg = 0.1
eta = 0.1 # learning rate
mu = np.mean(np.array(y_base + y_valid + y_hidden)[:, 3])
#U, V, a, b, rmse = train_model_with_bias(M, N, k, eta, reg, mu, np.array(y_base + y_valid + y_hidden + y_probe)[:, [0, 1, 3]])
U, V, a, b, rmse = train_model(M, N, k, eta, reg, np.array(y_base + y_valid + y_hidden)[:, [0, 1, 3]], mu)

V = V.T

Factorizing with  458293  users,  17770  movies.
0.4383475123973069
0.43347664091109556


KeyboardInterrupt: 

In [35]:
print("Test error is ", get_err(U, V.T,np.array(y_probe)[:, [0, 1, 3]], a, b, mu))

Test error is  [0.00081946]


In [36]:
test_preds = []
for (user, movie, rating) in np.array(y_qual)[:, [0, 1, 3]]:
    test_preds.append((mu + np.dot(U[user - 1], V.T[movie - 1]) + a[user - 1] + b[movie - 1])[0])

In [28]:
len(test_preds)

2749898

In [37]:
with open('test_preds2.txt', 'w') as f:
    for item in test_preds:
        f.write("%.4f\n" % item)